# In-Depth Introduction to Turing

This is an in-depth introduction to the [Turing.jl](https://turing.ml) a high-level probabilistic programming language written in [Julia](https://julialang.org). We will begin by building a simple model and then exploring the inference interface, before exploring the backend syntax of `@model`, the kinks and quirks of the automatic differentiation backend, how to express our probabilistic models performantly and visualizing our results

The tutorial is based on the [Turing tutorials](https://turing.ml/dev/docs/using-turing) and material from the [Zygote.jl](https://github.com/FluxML/Zygote.jl) [documentation](https://fluxml.ai/Zygote.jl/dev/) on automatic differentiation.


## Outline:

**Section 1** [Starting in Turing](#starting)

**Section 2** [Modelling Syntax](#modelling-syntax)

**Section 3** [Sampling with Multiple Chains](#multiple-chains)

**Section 4** [The Chain as a Data Structure](#chain-data-struct)

**Section 5** [Maximum Likelihood and Maximum a Posterior Estimates](#mle-map)

**Section 6** [Advanced Interface](#advanced)

**Section 7** [Automatic Differentation](#ad)

**Section 8** [Performance Optimization](#perf-opt)

**Section 9** [Sampler Visualization](#sampler-viz)

## Starting in Turing <a name="starting"></a>

We begin by importing our dependencies:

In [ ]:
using Turing
using StatsPlots
using Distributed

Distributions of random variables are defined by `x ~ distribution`. If `x` is already defined Turing views this as us conditioning on `x` having been drawn from this distribution. The likelihood is always computed using the `logpdf`.

We begin by defining a normal model with unknown mean and variance

In [ ]:
@model function gdemo(x, y)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    x ~ Normal(m, sqrt(s))
    y ~ Normal(m, sqrt(s))
end

Inference is performed using the `sample` statement to which we provide our probabilistic program as the first argument, and the sampler as the second argument. The available samplers are:
* Gibbs-sampler
* Hamiltonian Monte-Carlo sampler
* Hamiltonian Monte-Carlo sampler with Dual Averaging algorithm
* Importance sampling
* No-U-Turn sampler
* Particle Gibbs sampler
* Sequential Monte Carlo sampler

Using the `DynamicPPL.Sampler` interface we are furthermore able to implement our own inference algorithms for which we need to specify the type of the algorithm and parameters with inheritance from `InferenceAlgorithm` and a `sample` method.

In [ ]:
c1 = sample(gdemo(1.5, 2), SMC(), 1000)
c2 = sample(gdemo(1.5, 2), PG(10), 1000)
c3 = sample(gdemo(1.5, 2), HMC(0.1, 5), 1000)
c4 = sample(gdemo(1.5, 2), Gibbs(PG(10, :m), HMC(0.1, 5, :s)), 1000)
c5 = sample(gdemo(1.5, 2), HMCDA(0.15, 0.65), 1000)
c6 = sample(gdemo(1.5, 2), NUTS(0.65), 1000)

Where the APIs for each of the inference algorithms is defined as:

### Gibbs-Sampler

Args:
* multiple inference algorithms, which to be combined by the Gibbs sampler. This allows us to use particle-based methods, like `PG` for discrete variables and `HMC` for continuous variables.

### Hamiltonian Monte-Carlo

Args:
* leapfrog step size to use, which is to be reduced when receiving gradient errors
* number of leapfrog steps to use

### Hamiltonian Monte-Carlo Dual-Averaging

Args:
* Number of samples to use for adaptation
* Target acceptance rate, a good guess is usually 65%
* Target leapfrog length
* Initial step size, if the step size is 0 Turing automatically searches for the optimal step size

### Importance Sampling

Args:
* Requires no arguments

### No-U-Turn Sampler

Args:
* Number of samples to use with adaptation
* Target acceptance rate for dual averaging
* Maximum doubling tree depth
* Maximum divergence during double tree
* Initial step size, if the step size is 0 Turing automatically searches for the optimal step size

### Particle-Gibbs

Args:
* Number of particles
* Optionally a custom resampling algorithm

### Sequential Monte Carlo

Args:
* Requires no arguments, but can optionally specify a custom resampling algorithm

Viewing summary statistics for one of the sampled chains:

In [ ]:
describe(c3)

Plotting results:

In [ ]:
plot(c3)

## Modelling Syntax <a name="moelling-syntax"></a>

The order in which we write out the model code matters. In this example `s` needs to be declared before `y`, as `y`
uses the sampled result from `s`.

In [ ]:
@model function example(y)
    s ~ Poisson(1)
    y ~ Normal(s, 1)  # 
    return y
end

sample(example(10), SMC(), 100)

## Sampling with Multiple Chains <a name="multiple-chains"></a>

With Turing we employ threaded, as well as parallel sampling to fully exploit the computational resources available to use. This is defined in the `sample` statement with an extra argument inside of the call, i.e. `sample(model, sampler, {MCMCThreads(), MCMCDistributed()}, n, n_chains)`. With multiple chains we can then evaluate convergence characteristics. If parallelism is not desired, one can still sample multiple chains through the `mapreduce` function.

As the `chains` variable now contains multiple chains, which can be indexed by chain, e.g. for the first chain `chains[:, :, 1]`.

For multithreaded sampling with 4 chains in this example, for which the Julia instance needs to be started with multiple threads to actually "fill" those threads:

In [ ]:
@model function multithreading_demo(x)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

model = multithreading_demo([1.5, 2.0])

chain = sample(model, NUTS(), MCMCThreads(), 1000, 4; save_state=true)

Using distributed sampling with four sampling parallel sampling processes and enforcing a precompiled Turing on all processes.

In [ ]:
addprocs(4)

@everywhere using Turing

@everywhere @model function distributed_demo(x)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

@everywhere model = distributed_demo([1.5, 2.0])

#sample(model, NUTS(), MCMCDistributed(), 1000, 4) --> Takes way too long, bug?

To sample from a model's prior we can simply generate a chain out of the prior, i.e. `chain = sample(model, Prior(), n_samples)`. We are furthermore able to run our model from the prior distribution, by calling the model without specifying inputs or a sampler. Our example model returns two variables, but also takes two variables as input. Not specifying the two leads Turing to believe that they are missing values, which are to be drawn from the respective distribution.

In [ ]:
@model function unconditional_demo(x, y)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    x ~ Normal(m, sqrt(s))
    y ~ Normal(m, sqrt(s))
    return x, y
end

unconditional_prior_sample = unconditional_demo(missing, missing)
unconditional_prior_sample()

To sample from a model's posterior, i.e. treat observations as random variables we have to express our model
with the following syntax, where the missing variable needs to be initialized explicitly:

In [ ]:
@model function conditional_demo(x, ::Type{T} = Float64) where {T}
    if x === missing
        # Initialize `x` if missing
        x = Vector{T}(undef, 2)
    end
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

# Construct a model with x = missing
model = gdemo(missing)
c = sample(model, HMC(0.01, 5), 500)

It is also possible to use a mixture of `missing` and non-`missing` values in the same random variable `x`. The missing entries will be treated as random variables to be sampled, while non-`missing` values get treated as observations

In [ ]:
@model function conditional_missing_demo(x)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

model = conditional_missing_demo([missing, 2.4])
c = sample(model, HMC(0.01, 5), 500)

To treat an observation by default as a random variable, we can specify such trait in the model definition

In [ ]:
@model function conditional_default_demo(x = missing, ::Type{T} = Float64) where {T <: Real}
    if x === missing
        x = Vector{T}(undef, 10)
    end
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in 1:length(x)
        x[i] ~ Normal(m, sqrt(s))
    end
    return s, m
end

m = conditional_default_demo()
chain = sample(m, HMC(0.01, 5), 1000)

## The Chain as a Datastructure <a name="chain-data-struct"></a>

Values stored inside of a chain can be accessed in multiple ways:

1. Transform them into `DataFrames`
2. Use their raw `AxisArray` form
3. Create a three-dimensional `Array` object

When defining the variable types we need to abide by a few rules:

1. When using the Hamiltonian sampler we should use `real` values to enable auto-differentiation through the model
2. When using a particle sampler variables should preferably be 'TArray's
3. Use the type parameter definition in the model header

### Querying Probabilities from our Model, or our Chain

When having a demo model we can query the model and its likelihoods in a multitude of ways using the `prob` command, if we are interested in log-probabilities you just need to replace `prob` with `logprob` below.

In [ ]:
@model function probability_query_demo(x, y)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    x ~ Normal(m, sqrt(s))
    y ~ Normal(m, sqrt(s))
end

Calculate the likelihood of `x=1`and `y=1` given `s=1` and `m=1`

In [ ]:
prob"x=1.0, y=1.0 | model=probability_query_demo, s=1.0, m=1.0"

Calculate the joint probability of `s=1` and `m=1` ignoring `x` and `y`, so they can optionally be dropped.

In [ ]:
prob"s=1.0, m=1.0 | model=probability_query_demo, x=nothing, y=nothing"

Calculate the joint probability of `s=1`, `m=1` and `x=1` ignoring `y`

In [ ]:
prob"s=1.0, m=1.0, x=1.0 | model=probability_query_demo, y=nothing"

Calculate the joint probability of all variables

In [ ]:
prob"s=1.0, m=1.0, x=1.0, y=1.0 | model=probability_query_demo"

The same can be done for chains after MCMC sampling has taken place. E.g. calculating the element-wise likelihood of `x=1.0` and `y=1.0` for each sample in the chain `prob"x=1.0, y=1.0 | chain=chain, model=multithreading_demo"` or `prob"x=1.0, y=1.0 | chain=chain"` if `save_state=true` was set at sampling time.

## Maximum Likelihood and Maximum a Posterior Estimates <a name="mle-map"></a>

For mode estimation Turing provides maximum likelihood estimation (MLE) and maximum a posterior (MAP) estimation. To benefit from these capabilities, we need to load the `Optim` package. For mode estimation to work all parameters furthermore need to be continuous.

In [ ]:
using Optim

In [ ]:
@model function mode_demo(x)
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

In [ ]:
data = [1.5, 2.0]
model = mode_demo(data)

Benefitting from `Optim.optimize` Turing accepts `MLE()` or `MAP`. The optimizer uses LBFGS by default.

In [ ]:
mle_estimate = optimize(model, MLE())
map_estimate = optimize(model, MAP())

The optimizer can be altered by using the optional third argument and putting an alternative such as

* NelderMead
* SimulatedAnnealing
* ParticleSwarm
* Newton
* AcceleratedGradientDescent

If Optim fails to converge, it is prudent to increase the number of possible `iterations` or allow for steps that increase the objective value by setting `allow_f_increase` to true in an additional argument. This would then look like this:

In [ ]:
mle_estimate = optimize(model, MLE(), Newton(), Optim.Options(iterations=10_000, allow_f_increases=true))

Turing benefits from inheritance from the `StatsBase` package, which provides analysis tools to dissect the mode estimation results. The most prevalent of which are coefficient-tables (`coeftable`), and the Fisher information matrix (`informationmatrix`). E.g. using the coefficient table

In [ ]:
using StatsBase

In [ ]:
coeftable(mle_estimate)

We can also begin sampling a chain from the MLE/MAP estimate. For this we have to extract the vector of parameter values and provide it to the `sample` function to `init_theta`. E.g. sampling from the full posterior using the MAP estimate

In [ ]:
map_estimate = optimize(model, MAP())
chain = sample(model, NUTS(), 1_000, init_theta=map_estimate.values.array)

### Diagnostics of Chains using MCMCChains.jl

Turing's samples are wrapped by `MCMCChains.Chain`, hence allowing the analysis of chains using all available diagnostics from [MCMCChains](https://github.com/TuringLang/MCMCChains.jl). Examples of available functionality include, but are not limited to:

Convergence diagnostics such as:
* Gelman, Rubin, and Brooks Diagnostics
* Geweke Diagnostics
* Heidelberger and Welch Diagnostics
* Raftery and Lewis Diagnostics

Model selection with the *Deviance Information Criterion (DIC)*, plotting support and multiple ways to export chains into `Array` structures or `DataFrame` structures.

## Advanced Interfaces <a name="advanced"></a>

### Customized Distributions

While Turing support a wide array of distributions through the [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) package, it allows for the custom definition of distributions by benefitting from subtypes of `Distribution`, which then have to be complemented with corresponding functions. E.g. if we were to define a uniform distribution it would look like this:

In [ ]:
struct CustomUniform <: ContinuousUnivariateDistribution
end

# Define rand and logpdf
Distributions.rand(rng::AbstractRNG, d::CustomUniform) = rand(rng)
Distributions.logpdf(d::CustomUniform, x::Real) = zero(x)

As `HMC` requires the domain of priors to be unbounded we furthermore need to define a bijector from `[0, 1]` to `ℝ`. For this we have to use the [`Bijectors.jl`](https://github.com/TuringLang/Bijectors.jl) package, i.e.

In [ ]:
Bijectors.bijector(d::CustomUniform) = Logit(0., 1.)

In the uniform case we furthermore want to define the minimum and maximum

In [ ]:
Distributions.minimum(d::CustomUniform) = 0.
Distributions.maximum(d::CustomUniform) = 1.

For performance-oriented implementations it is furthermore recommended to implement vectorization support.

### Model Internals

`@model` generates a `Turing.Model` structure, which is then re-used by the sampler. We will now examine what the `gdemo` example translates into when the macro is not used.

In [ ]:
@model gdemo(x) = begin
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    
    # Observe each value of x
    @. x ~ Normal(m, sqrt(s))
end

sample(gdemo([1.5, 2.0]), HMC(0.1, 5), 1000)

In [ ]:
# Initialize a NamedTuple containing the data variables
data = (x = [1.5, 2.0],)

# Create model function
mf(vi, sampler, ctx, model) = begin
    # Set the accumulated logp to zero
    resetlogp!(vi)
    x = model.args.x
    
    # Assume s has an InverseGamma distribution
    s, lp = Turing.Inference.tilde(
        ctx,
        sampler,
        InverseGamma(2, 3),
        Turing.@varname(s),
        (),
        vi,
    )
    
    # Add the lp to the accumullated logp
    acclogp!(vi, lp)
    
    # Assume m has a Normal distribution
    m, lp = Turing.Inference.tilde(
        ctx,
        sampler,
        Normal(0, sqrt(s)),
        Turing.@varname(m),
        (),
        vi,
    )
    
    # Add the lp to the accumulated logp
    acclogp!(vi, lp)
    
    # Observe each value of x[i], according to a
    # Normal distribution
    lp = Turing.Inference.dot_tilde(ctx, sampler, Normal(m, sqrt(s)), x, vi)
    acclogp!(vi, lp)
end

# Instantiate the model object
model = DynamicPPL.Model(mf, data, DynamicPPL.ModelGen{()}(nothing, nothing))

# Sample the model
chain = sample(model, HMC(0.1, 5), 1000)

## Automatic Differentiation <a name="ad"></a>

### Introduction

Zygote's core ingredient is the definition of the pullback, which defines the adjoint for the computational operation. All automatic differentiation frameworks, have to perform this step on some level. Looking at this in Zygote:

In [ ]:
using Zygote

In [ ]:
y, back = Zygote.pullback(sin, 0.5)

`back` implements the pullback, which is the gradient computation for `sin` here. It implements a vector-Jacobian product, where ``y = f(x)`` and the gradient is written ``x-bar``, the pullback ``\mathcal{B}_y`` then computer the partial of l over the partial of x, which is the same as the partial of l w.r.t. y times the partial of y w.r.t. x, which is the same as the pullback.

`pullback(sin, x)` is hence the same as

In [ ]:
dsin(x) = (sin(x), ȳ -> (ȳ * cos(x),))

Which generally expressed then amounts to 

In [ ]:
function mygradient(f, x...)
    _, back = Zygote.pullback(f, x...)
    back(1)
end

mygradient(sin, 0.5)

### Compositional Sampling with Differing AD Modes

Within Julia there currently exist a number of different automatic differentiation backends, the most famous of which `Zygote.jl` is viewed as the future, but has so far not come out of alpha-stage yet. This has hence lead to most performant codes relying on a mixture of automatic differentiation frameworks. Turing, as well as Gen for that matter, does by default rely on `ForwardDiff`, but has the ability to switch its backend to `Tracker`, `Zygote` or `ReverseDiff` who can incur their own performance penalties, but mixed-use is also possible.

One would switch the backend by invoking `Turing.setadbackend({:forwarddiff, :tracker, :zygote, :reversediff})`

Attention: `ReverseDiff` invites the use of [memoization](https://en.wikipedia.org/wiki/Memoization), caching already sampled values for later reuse, which can result in incorrect results or crashes in some probabilistic programs.

In [ ]:
@model compositional_sampling_demo(x, y) = begin
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    x ~ Normal(m, sqrt(s))
    y ~ Normal(m, sqrt(s))
end

c = sample(
    compositional_sampling_demo(1.5, 2),
    Gibbs(
        HMC{Turing.ForwardDiffAD{1}}(0.1, 5, :m),
        HMC{Turing.TrackerAD}(0.1, 5, :s)
    ),
    1000
)

The different backends all have their own advantages, which can used to boost performance. `TrackerAD` is best-suited for high dimensional problems, whereas `ForwardDiffAD` shines on lower-dimensional variables. The default backend is `ForwardDiffAD`. If you decide to use `Tracker` or `Zygote` loops should be avoided at all costs, as they incur heavy performance penalties. Loops can be avoided by utilizing matrix-variate distributions or by writing custom distribution with corresponding custom adjoints.

## Performance Optimization <a name="perf-opt"></a>

Loops inside of the generative model should be avoided, try to replace loops with multivariate distributions when possible. E.g.

In [ ]:
@model multivariate_demo(x) = begin
    m ~ Normal()
    for i = 1:length(x)
        x[i] ~ Normal(m, 0.2)
    end
end

In [ ]:
@model multivariate_demo(x) = begin
    m ~ Normal()
    x ~ MvNormal(fill(m, length(x)), 0.2)
end

Ensure type-stability of your probabilistic models. This ensures that Julia always returns a value of the same type as output, improves performance i.e. how many functions does Julia need to compile for an expression? It furthermore makes life easier for the inference algorithms. `@code_warntype` can furthermore unearth type instabilities in the model definitions.

Not type-stable:

In [ ]:
@model tmodel(x, y) = begin
    p, n = size(x)
    params = Vector{Real}(undef, n)
    for i = 1:n
        params[i] ~ truncated(Normal(), 0, Inf)
    end
    a = x * params
    y ~ MvNormal(a, 1.0)
end

Type-stable:

In [ ]:
@model tmodel(x, y, ::Type{T}=Vector{Float64}) where {T} = begin
    p, n = size(x)
    params = T(undef, n)
    for i = 1:n
        params[i] ~ truncated(Normal(), 0, Inf)
    end
    a = x * params
    y ~ MvNormal(a, 1.0)
end

Example for an effective use of `@code_warntype`:

In [ ]:
@model tmodel(x) = begin
    p = Vector{Real}(undef, 1);
    p[1] ~ Normal()
    p = p .+ 1
    x ~ Normal(p[1])
end

model = tmodel(1.0)
varinfo = Turing.VarInfo(model)
spl = Turing.SampleFromPrior()

@code_warntype model.f(model, varinfo, spl, Turing.DefaultContext())

A core concept, which when used properly can be highly benefitial in scientific computing application is the one of memoization, for which we need use [Memoization.jl](https://github.com/marius311/Memoization.jl) in Julia with which we can then cache expensive function evaluations at inference time, especially in the context Gibbs sampling with all of its sub-iterations. Memoization essentially stores a function execution, so we we can reuse the output for every input it has already been evaluated with. For this we define memoized version of the function executions, which we can potentially identify through profiling.

Non-memoized version:

In [ ]:
@model memoization_demo(x) = begin
    a ~ Gamma()
    b ~ Normal()
    c = function1(a)
    d = function2(b)
    x .~ Normal(c, d)
end

alg = Gibbs(MH(:a), MH(:b))
sample(memoization_demo(zeros(10)), alg, 1000)

Memoized version:

In [ ]:
using Memoization

# define memoized function versions
@memoize memoized_function1(args...) = function1(args...)
@memoize memoized_function2(args...) = function2(args...)

In [ ]:
@model memoized_demo(x) = begin
    a ~ Gamma()
    b ~ Normal()
    c = memoized_function1(a)
    d = memoized_function2(b)
    x .~ Normal(c, d)
end

## Sampler Visualization <a name="sampler-viz"></a>

Set up a plotting function to visualize the sampler paths by plotting the sampler's trajectory across the posterior.

In [ ]:
ENV["GKS_ENCODING"] = "utf-8"
using Plots
using StatsPlots
using Turing
using Bijectors
using Random
using DynamicPPL: getlogp, settrans!, getval, reconstruct, vectorize, setval!

In [ ]:
# Set seed for reproducibility
Random.seed!(0)

@model viz_demo(x) = begin
    s ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s))
    bumps = sin(m) + cos(m)
    m = m + 5*bumps
    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s))
    end
    return s, m
end

# Define data points
x = [1.5, 2.0, 13.0, 2.1, 0.0]

model = viz_demo(x)
vi = Turing.VarInfo(model)

In [ ]:
# Convert variance parameters to the real prior to sampling
dist = InverseGamma(2,3)
svn = vi.metadata.s.vns[1]
mvn = vi.metadata.m.vns[1]
setval!(vi, vectorize(dist, Bijectors.link(dist, reconstruct(dist, getval(vi, svn)))), svn)
settrans!(vi, true, svn)

# Evaluate surface at coordinates
function evaluate(m1, m2)
    spl = Turing.SampleFromPrior()
    vi[svn] = [m1]
    vi[mvn] = [m2]
    model(vi, spl)
    getlogp(vi)
end

In [ ]:
function plot_sampler(chain; label="")
    # Extract values from chain.
    val = get(chain, [:s, :m, :lp])
    ss = link.(Ref(InverseGamma(2, 3)), val.s)
    ms = val.m
    lps = val.lp

    # How many surface points to sample.
    granularity = 100

    # Range start/stop points.
    spread = 0.5
    σ_start = minimum(ss) - spread * std(ss); σ_stop = maximum(ss) + spread * std(ss);
    μ_start = minimum(ms) - spread * std(ms); μ_stop = maximum(ms) + spread * std(ms);
    σ_rng = collect(range(σ_start, stop=σ_stop, length=granularity))
    μ_rng = collect(range(μ_start, stop=μ_stop, length=granularity))

    # Make surface plot.
    p = surface(σ_rng, μ_rng, evaluate,
          camera=(30, 65),
        #   ticks=nothing,
          colorbar=false,
          color=:inferno,
          title=label)

    line_range = 1:length(ms)

    scatter3d!(ss[line_range], ms[line_range], lps[line_range],
        mc =:viridis, marker_z=collect(line_range), msw=0,
        legend=false, colorbar=false, alpha=0.5,
        xlabel="σ", ylabel="μ", zlabel="Log probability",
        title=label)

    return p
end;

### Gibbs Sampler

In [ ]:
c = sample(model, Gibbs(HMC(0.01, 5, :s), PG(20, :m)), 1000)
plot_sampler(c)

### Hamiltonian Monte-Carlo

In [ ]:
c = sample(model, HMC(0.01, 10), 1000)
plot_sampler(c)

### Hamiltonian Monte-Carlo with Dual Averaging

In [ ]:
c = sample(model, HMCDA(200, 0.65, 0.3), 1000)
plot_sampler(c)

### No U-Turn Sampler (NUTS)

In [ ]:
c = sample(model, NUTS(0.65), 1000)
plot_sampler(c)

In [ ]:
c = sample(model, NUTS(0.95), 1000)
plot_sampler(c)

In [ ]:
c = sample(model, NUTS(0.2), 1000)
plot_sampler(c)

### Particle Gibbs sampler

In [ ]:
c = sample(model, PG(20), 1000)
plot_sampler(c)

In [ ]:
c = sample(model, PG(50), 1000)
plot_sampler(c)